In [1]:
import gensim
import csv
import logging
import nltk
import numpy as np

In [2]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

In [3]:
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)  

2017-03-07 19:09:17,891 : INFO : loading projection weights from GoogleNews-vectors-negative300.bin.gz
2017-03-07 19:11:47,115 : INFO : loaded (3000000, 300) matrix from GoogleNews-vectors-negative300.bin.gz


In [4]:
datafile = "data/sarcasm_v2.csv"
with open(datafile) as f:
    data = list(csv.reader(f))[0:]

In [27]:
text = [line[-1].lower() for line in data if line[0]=="GEN"]

In [7]:
tokens = [nltk.word_tokenize(line) for line in text]
print(tokens[0])

['therefore', 'you', 'accept', 'that', 'the', 'republican', 'party', 'almost', 'as', 'a', 'whole', 'is', '``', 'grade', 'a', 'usda', 'approved', 'liberalism', '.', "''", 'about', 'time', 'you', 'did', '.']


In [28]:
text = [[word for word in line if word not in nltk.corpus.stopwords.words('english')] for line in tokens]

In [11]:
text[0]

['therefore',
 'accept',
 'republican',
 'party',
 'almost',
 'whole',
 '``',
 'grade',
 'usda',
 'approved',
 'liberalism',
 '.',
 "''",
 'time',
 '.']

In [43]:
model.doesnt_match('cat dog sink cow'. split())

'sink'

In [18]:
model_name = "word2vec"
model.save(model_name)

In [19]:
model.syn0.shape


(3000000, 300)

In [18]:
def makeFeatureVec(doc,model):
    featureVec = np.zeros(300,)
    
    nwords = 0
    
    index2word_set = set(model.index2word)
    
    for word in doc:
        if word in index2word_set:
            nwords += 1
            featureVec = np.add(featureVec, model[word])
    
    featureVec = np.divide(featureVec, nwords)
    return featureVec
    

In [ ]:
model.vocab

In [19]:
def getAvgFeatureVecs(text, model):
    counter = 0
    
    textFeatureVecs = np.zeros((len(text), 300), dtype="float32")
    for doc in text:
#         if counter%1000 == 0:
#             print(type(counter), type(len(text)))
#             print("doc %d of %d") %(counter, len(doc))
            
        textFeatureVecs[counter] = makeFeatureVec(doc, model)
        counter += 1
    return textFeatureVecs
    

In [21]:
test = getAvgFeatureVecs(text[:10], model)
test.shape

(10, 300)

In [17]:
test.shape

(10, 300)

In [69]:
featureVec = np.zeros((1, 300),dtype="float32")

for doc in text[:2]:
    nwords = 0
    for word in doc:
        if word in set(model.index2word):
            nwords += 1
            featureVec = np.add(featureVec,model[word])
    featureVec = np.divide(featureVec, nwords)
            
            

In [32]:
featureVec

array([[-0.18475981,  0.13650087, -0.01074219, ..., -0.03607887,
        -0.09047113,  0.16226836],
       [-0.18475981,  0.13650087, -0.01074219, ..., -0.03607887,
        -0.09047113,  0.16226836],
       [-0.18475981,  0.13650087, -0.01074219, ..., -0.03607887,
        -0.09047113,  0.16226836],
       ..., 
       [-0.18475981,  0.13650087, -0.01074219, ..., -0.03607887,
        -0.09047113,  0.16226836],
       [-0.18475981,  0.13650087, -0.01074219, ..., -0.03607887,
        -0.09047113,  0.16226836],
       [-0.18475981,  0.13650087, -0.01074219, ..., -0.03607887,
        -0.09047113,  0.16226836]], dtype=float32)

In [22]:
from sklearn.feature_extraction.text import CountVectorizer

In [23]:
cv = CountVectorizer()

In [29]:
text_vocab = cv.fit_transform(text)

AttributeError: 'list' object has no attribute 'lower'

In [42]:
text_vocab.shape

(3260, 11663)

In [47]:
from sklearn.feature_extraction.text import TfidfTransformer as TfIdf

In [50]:
tf_idf = TfIdf()
tf_idf_features = tf_idf.fit(text_vocab)